In [ ]:
Question Answering Chatbot overDocuments_with_Sources

### Installing Required Libraries

In [1]:
!pip install langchain==0.1.4 deeplake openai==0.27.8 tiktoken

### Installing newspaper3k for extracting and curating articles from news websites

In [2]:
!pip install -q newspaper3k==0.2.8

ERROR: Operation cancelled by user


### Setting up OPENAI_API_KEY & ACTIVELOOP_TOKEN

In [2]:
import os
os.environ['OPENAI_API_KEY']='sk-proj-K74kvdsdfkl;ds;XnXovwvk5OCUFVxIm'

os.environ['ACTIVELOOP_TOKEN']='eyJhbGciOiJub2dfsl;kdsaNpZnRhaW4iLCJhcGlfa2Vdsk;ldsk;kltVGxEVUdTUDcwTlNEWGRMbzIzcVVWTjJlIn0.'
# Don't try api key, i have changed it 😂😂😂

### Imporitng requests for making HTTP requests

In [3]:
import requests
from newspaper import Article # https://github.com/codelucas/newspaper
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_urls = [
    "https://www.artificialintelligence-news.com/2023/05/16/openai-ceo-ai-regulation-is-essential/",
    "https://www.artificialintelligence-news.com/2023/05/15/jay-migliaccio-ibm-watson-on-leveraging-ai-to-improve-productivity/",
    "https://www.artificialintelligence-news.com/2023/05/15/iurii-milovanov-softserve-how-ai-ml-is-helping-boost-innovation-and-personalisation/",
    "https://www.artificialintelligence-news.com/2023/05/11/ai-and-big-data-expo-north-america-begins-in-less-than-one-week/",
    "https://www.artificialintelligence-news.com/2023/05/02/ai-godfather-warns-dangers-and-quits-google/",
    "https://www.artificialintelligence-news.com/2023/04/28/palantir-demos-how-ai-can-used-military/"
]

session = requests.Session()
pages_content = [] # where we save the scraped articles

for url in article_urls:
    try:
        time.sleep(2) # sleep two seconds for gentle scraping
        response = session.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            article = Article(url)
            article.download() # download HTML of webpage
            article.parse() # parse HTML to extract the article text
            pages_content.append({ "url": url, "text": article.text })
        else:
            print(f"Failed to fetch article at {url}")
    except Exception as e:
        print(f"Error occurred while fetching article at {url}: {e}")

#If an error occurs while fetching an article, we catch the exception and print
#an error message. This ensures that even if one article fails to download,
#the rest of the articles can still be processed.

### Computing Embeddings and Storing in Deep Lake

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

my_activeloop_org_id = "siftain" # TODO: use your organization id here
my_activeloop_dataset_name = "langchain_course_qabot_with_source"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Your Deep Lake dataset has been successfully created!


### Splitting the article texts into small chunks using RecursiveCharacterTextSplitter

In [5]:
# We split the article texts into small chunks. While doing so, we keep track of each
# chunk metadata (i.e. the URL where it comes from). Each metadata is a dictionary and
# we need to use the "source" key for the document source so that we can then use the
# RetrievalQAWithSourcesChain class which will automatically retrieve the "source" item
# from the metadata dictionary.

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

all_texts, all_metadatas = [], []
for d in pages_content:
    chunks = text_splitter.split_text(d["text"])
    for chunk in chunks:
        all_texts.append(chunk)
        all_metadatas.append({ "source": d["url"] })

### Adding the chunks to Deep Lake

In [6]:
# we add all the chunks to the deep lake, along with their metadata
db.add_texts(all_texts, all_metadatas)

Creating 49 embeddings in 1 batches of size 49:: 100%|██████████| 1/1 [00:13<00:00, 13.99s/it]

Dataset(path='hub://siftain/langchain_course_qabot_with_source', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (49, 1)      str     None   
 metadata     json      (49, 1)      str     None   
 embedding  embedding  (49, 1536)  float32   None   
    id        text      (49, 1)      str     None   


['bbb10a3e-2944-11ef-834e-0242ac1c000c',
 'bbb10c0a-2944-11ef-834e-0242ac1c000c',
 'bbb10ce6-2944-11ef-834e-0242ac1c000c',
 'bbb10da4-2944-11ef-834e-0242ac1c000c',
 'bbb10e58-2944-11ef-834e-0242ac1c000c',
 'bbb10ef8-2944-11ef-834e-0242ac1c000c',
 'bbb10fac-2944-11ef-834e-0242ac1c000c',
 'bbb1104c-2944-11ef-834e-0242ac1c000c',
 'bbb110e2-2944-11ef-834e-0242ac1c000c',
 'bbb1118c-2944-11ef-834e-0242ac1c000c',
 'bbb11222-2944-11ef-834e-0242ac1c000c',
 'bbb112c2-2944-11ef-834e-0242ac1c000c',
 'bbb113da-2944-11ef-834e-0242ac1c000c',
 'bbb1148e-2944-11ef-834e-0242ac1c000c',
 'bbb11538-2944-11ef-834e-0242ac1c000c',
 'bbb115d8-2944-11ef-834e-0242ac1c000c',
 'bbb11678-2944-11ef-834e-0242ac1c000c',
 'bbb1172c-2944-11ef-834e-0242ac1c000c',
 'bbb117ea-2944-11ef-834e-0242ac1c000c',
 'bbb1188a-2944-11ef-834e-0242ac1c000c',
 'bbb1192a-2944-11ef-834e-0242ac1c000c',
 'bbb119ca-2944-11ef-834e-0242ac1c000c',
 'bbb11a74-2944-11ef-834e-0242ac1c000c',
 'bbb11b1e-2944-11ef-834e-0242ac1c000c',
 'bbb11bc8-2944-

### Building the QA Chatbot which also keeps track of the sources using RetrievalQAWithSourcesChain

In [7]:
# we create a RetrievalQAWithSourcesChain chain, which is very similar to a
# standard retrieval QA chain but it also keeps track of the sources of the
# retrieved documents

from langchain.chains import RetrievalQAWithSourcesChain
from langchain import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                                    chain_type="stuff",
                                                    retriever=db.as_retriever())

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


### Generating a Response along with it's source/sources

In [8]:
# We generate a response to a query using the chain. The response object is a dictionary containing
# an "answer" field with the textual answer to the query, and a "sources" field containing a string made
# of the concatenation of the metadata["source"] strings of the retrieved documents.
d_response = chain({"question": "What does Geoffrey Hinton think about recent trends in AI?"})

print("Response:")
print(d_response["answer"])
print("Sources:")
for source in d_response["sources"].split(", "):
    print("- " + source)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response:
 Geoffrey Hinton, known as the "Godfather of AI," has expressed concerns about the potential dangers of AI and left his position at Google to discuss them openly. He has warned about the rapid development of generative AI products and the potential for false information to be spread. Hinton also expressed concerns about the impact of AI on the job market. Other experts, such as Elon Musk, Neil deGrasse Tyson, and Stephen Hawking, have also warned about the risks of AI. 

Sources:
- https://www.artificialintelligence-news.com/2023/05/02/ai-godfather-warns-dangers-and-quits-google/


# Thank You